Based on https://github.com/MedMNIST/MedMNIST/blob/main/examples/getting_started.ipynb

Env preparation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/MLMI

/content/drive/MyDrive/MLMI


In [ ]:
!pip install medmnist torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 8.3 MB/s 
     |████████████████████████████████| 87 kB 6.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=524f811185ef78fb6cfc40c152edac6fce4d062c55574332485f153f8a1a1b83
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire


In [ ]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F
import copy
from sklearn.metrics import accuracy_score

import os


import medmnist
from medmnist import INFO, Evaluator

from __future__ import print_function, division

import random
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import copy

import os
from os import listdir
from os.path import join

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F
from torch import logsumexp, zeros_like, ones_like

import torchvision
import torchvision.utils as vutils
from torchvision import utils, models

np.random.seed(42)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)

In [ ]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v2.1.0 @ https://github.com/MedMNIST/MedMNIST/


Parameters

In [ ]:
data_flag = 'pathmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 100
BATCH_SIZE = 64
PATIENCE = 30
lr = 0.002
weight_decay = 0.001
number_of_res_blocks = 5

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])
task

'multi-class'

In [ ]:
def create_labeled_subset(dataset, n_classes=9, per_class = 1000):
  label_buckets = [[] for _ in range(0, n_classes)]
  for img, label in dataset:
    label_buckets[label.item()].append((img, label))

  label_sub_bucket = [[] for _ in range(0, n_classes)]
  
  for i, bucket in enumerate(label_buckets):
    label_sub_bucket[i] = random.sample(bucket, per_class)

  balanced_labelled = []
  for b in label_sub_bucket:
    balanced_labelled.extend(b)

  return data.DataLoader(balanced_labelled, batch_size=BATCH_SIZE, shuffle=True)


MedMNIST data, preprocessing, data loader


In [ ]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
val_dataset = DataClass(split='val', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader= data.DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)



from torch.utils.data import Subset
unlabeled_subset = Subset(train_dataset, range(int(len(train_dataset) * 0.5)))
unlabeled_loader = data.DataLoader(dataset=unlabeled_subset, batch_size=BATCH_SIZE, shuffle=True)

labeled_loader = create_labeled_subset(train_dataset, per_class=1000)

Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz
Using downloaded and verified file: /root/.medmnist/pathmnist.npz


In [ ]:
# visualization

train_dataset.montage(length=1)

In [ ]:
# montage
m = train_dataset.montage(length=10)
m.save('train.png')

ResNet model definition


In [ ]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [ ]:
class ResidualBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()      
        self.info = 'in ' +  str(in_channels) + ', out ' + str(out_channels) + ', stride ' + str(stride)
        if stride > 1 or in_channels != out_channels:
            # Add strides in the skip connection and zeros for the new channels.
            self.skip = Lambda(lambda x: F.pad(x[:, :, ::stride, ::stride], (0, 0, 0, 0, 0, out_channels - in_channels), mode="constant", value=0))
        else:
            self.skip = nn.Sequential()
 
        conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        bn1 = nn.BatchNorm2d(out_channels)
        conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding='same')
        bn2 = nn.BatchNorm2d(out_channels)
        self.l1 = nn.Sequential(conv1, bn1)
        self.l2 = nn.Sequential(conv2, bn2)

    def forward(self, input):
        skip = self.skip(input)
        x = self.l1(input)
        x = F.relu(x)
        x = self.l2(x)
        return F.relu(x + skip)

In [ ]:
class ResidualStack(nn.Module):
    
    def __init__(self, in_channels, out_channels, stride, num_blocks):
        super().__init__()
        first = [ResidualBlock(in_channels, out_channels, stride)]
        rest = [ResidualBlock(out_channels, out_channels) for i in range(num_blocks - 1)]
        self.modules_list = nn.Sequential(*(first + rest))
        
    def forward(self, input):
        return self.modules_list(input)

Train and evaluate

In [ ]:
def initialize_weight(module):
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
    elif isinstance(module, nn.BatchNorm2d):
        nn.init.constant_(module.weight, 1)
        nn.init.constant_(module.bias, 0)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda


In [ ]:
def run_epoch(model, optimizer, dataloader, criterion):
  
    model.train()

    epoch_loss = 0.0
    epoch_acc = 0.0

    for inputs, targets in tqdm(dataloader):
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        
        if task == 'multi-label, binary-class':
            targets = targets.to(torch.float32)
        else:
            targets = targets.squeeze().long()

        loss = criterion(outputs, targets)
            
        top1 = torch.argmax(outputs, dim=1).long()
        ncorrect = torch.sum(top1 == targets)
        epoch_loss += loss.item()
        epoch_acc += ncorrect.item()

        loss.backward()
        optimizer.step()

    epoch_loss /= len(dataloader.dataset)
    epoch_acc /= len(dataloader.dataset)

    return epoch_loss, epoch_acc

In [ ]:
best_model_weights = None

def fit(model, optimizer, lr_scheduler, task):

    if task == "multi-label, binary-class":
      criterion = nn.BCEWithLogitsLoss()
    else:
      criterion = nn.CrossEntropyLoss()
    
    _ = model.to(device)
    _ = model.apply(initialize_weight)
    
    best_acc = 0.0
    curr_patience = 0.0
    min_val_loss = 1.0

    val_losses = []
    val_accuracies = []
    train_losses = []
    train_accuracies = []

    for epoch in range(NUM_EPOCHS):

      train_loss, train_acc = run_epoch(model, optimizer, labeled_loader, criterion)
      lr_scheduler.step()
      print(f"Epoch {epoch + 1: >3}/{NUM_EPOCHS}, train loss: {train_loss:.2e}, train acc: {train_acc:.3f}")

      train_losses.append(train_loss)
      train_accuracies.append(train_acc)
      
      val_loss, val_acc, val_top5 = test(model, val_loader, 'val', criterion)
      print(f"Epoch {epoch + 1: >3}/{NUM_EPOCHS}, val loss: {val_loss:.2e}, val acc: {val_acc:.3f}, val top5: {val_top5:.3f}")

      val_losses.append(val_loss)
      val_accuracies.append(val_acc)

      if val_loss >= min_val_loss:
        curr_patience += 1
        if curr_patience == PATIENCE:
          break
      else:
        curr_patience = 0
        min_val_loss = val_loss
        best_model_weights = copy.deepcopy(model.state_dict())
        torch.save(model.state_dict(), 'ResNet.pt')

    model.load_state_dict(best_model_weights)

    plt.plot(range(len(val_losses)), np.array(val_losses))
    plt.title('val loss')
    plt.show()

    plt.plot(range(len(val_accuracies)), np.array(val_accuracies))
    plt.title('val acc')
    plt.show()

    plt.plot(range(len(train_losses)), np.array(train_losses))
    plt.title('train loss')
    plt.show()

    plt.plot(range(len(train_accuracies)), np.array(train_accuracies))
    plt.title('train acc')
    plt.show()

In [ ]:
def test(model, data_loader, mode, criterion):
    model.eval()
    epoch_acc = 0.0
    epoch_acc_top5 = 0.0
    epoch_loss = 0.0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
            else:
                targets = targets.squeeze().long()

            loss = criterion(outputs, targets)

            top1 = torch.argmax(outputs, dim=1).long()
            ncorrect = torch.sum(top1 == targets)

            _ , top5 = torch.topk(outputs, 5, dim=1)
            top5 = top5.t()
            correct = top5.eq(targets.reshape(1, -1).expand_as(top5))
            ncorrect_top5 = correct[:5].reshape(-1).float().sum(0, keepdim=True)

            epoch_loss += loss.item()
            epoch_acc += ncorrect.item()
            epoch_acc_top5 += ncorrect_top5.item()

        
        epoch_loss /= len(data_loader.dataset)
        epoch_acc /= len(data_loader.dataset)  
        epoch_acc_top5 /= len(data_loader.dataset)  
    
    return epoch_loss, epoch_acc, epoch_acc_top5

In [ ]:
if task == 'multi-label, binary-class':
  criterion = nn.BCEWithLogitsLoss()
else:
  criterion = nn.CrossEntropyLoss()

criterion

CrossEntropyLoss()

In [ ]:
res_net_18 = nn.Sequential(
          nn.Conv2d(n_channels, 64, kernel_size=7, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          ResidualStack(64, 64, 1, 6),
          ResidualStack(64, 128, 2, 8),
          ResidualStack(128, 256, 2, 12),
          ResidualStack(256, 512, 2, 6),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
          nn.Linear(512, n_classes)
      )

In [ ]:
optimizer = optim.Adam(res_net_18.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 80], gamma=0.1)

# fit(res_net_18, optimizer, lr_scheduler, task) 

In [ ]:
# test(res_net_18, test_loader, 'test', criterion)

In [ ]:
# test(res_net_18, val_loader, 'val', criterion)

In [ ]:
def test(model, data_loader, mode, criterion):
    model.eval()
    epoch_acc = 0.0
    epoch_acc_top5 = 0.0
    epoch_loss = 0.0

    with torch.no_grad():
        for inputs, targets in data_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            _, outputs = model(inputs)

            if task == 'multi-label, binary-class':
                targets = targets.to(torch.float32)
            else:
                targets = targets.squeeze().long()

            loss = criterion(outputs, targets)

            top1 = torch.argmax(outputs, dim=1).long()
            ncorrect = torch.sum(top1 == targets)

            _ , top5 = torch.topk(outputs, 5, dim=1)
            top5 = top5.t()
            correct = top5.eq(targets.reshape(1, -1).expand_as(top5))
            ncorrect_top5 = correct[:5].reshape(-1).float().sum(0, keepdim=True)

            epoch_loss += loss.item()
            epoch_acc += ncorrect.item()
            epoch_acc_top5 += ncorrect_top5.item()

        
        epoch_loss /= len(data_loader.dataset)
        epoch_acc /= len(data_loader.dataset)  
        epoch_acc_top5 /= len(data_loader.dataset)  
    
    return epoch_loss, epoch_acc, epoch_acc_top5

SGAN
based on https://github.com/opetrova/SemiSupervisedPytorchGAN/blob/master/SemiSupervisedGAN.ipynb and https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/sgan/sgan.py


In [ ]:
FloatTensor = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if torch.cuda.is_available() else torch.LongTensor

class Discriminator(nn.Module):
    def __init__(self, num_classes):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
          nn.Conv2d(n_channels, 64, kernel_size=7, stride=1, padding='same', bias=False),
          nn.BatchNorm2d(64),
          nn.ReLU(),
          ResidualStack(64, 64, 1, 6),
          ResidualStack(64, 128, 2, 8),
          ResidualStack(128, 256, 2, 12),
          ResidualStack(256, 512, 2, 6),
          nn.AdaptiveAvgPool2d(1),
          Lambda(lambda x: x.squeeze()),
      )
    
        # Output layers
        self.adv_layer = nn.Sequential(nn.Linear(512, 1))
        self.aux_layer = nn.Sequential(nn.Linear(512, num_classes))

    def forward(self, img):
        out = self.main(img)
        # out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label   

class Generator(nn.Module):
    def __init__(self, num_classes, latent_dim, img_size, channels):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(num_classes, latent_dim)

        self.init_size = img_size // 4  # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, noise):
        out = self.l1(noise)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

# Number of channels in the training images. For color images this is 3
nc = n_channels
# Size of z latent vector 
nz = 50
# Size of feature maps in generator
ngf = 64

num_classes = 9

# A noise vector to be used for generating images at the end of each training epoch
fixed_noise = torch.randn(ngf, nz, device=device)

generator = Generator(num_classes, nz, 28, n_channels).to(device)
# generator.apply(initialize_weight)
discriminator = Discriminator(num_classes).to(device)
discriminator.apply(initialize_weight)

generator.load_state_dict(torch.load('Gan_G_copy50.pt'))
# discriminator.load_state_dict(torch.load('Gan_D_copy50.pt'))

optimizer_G = optim.Adam(generator.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler_G = torch.optim.lr_scheduler.MultiStepLR(optimizer_G, milestones=[10, 50], gamma=0.1)

optimizer_D = optim.Adam(discriminator.parameters(), lr=lr, weight_decay=weight_decay)
lr_scheduler_D = torch.optim.lr_scheduler.MultiStepLR(optimizer_D, milestones=[10, 50], gamma=0.1)

val_losses = []
val_accuracies = []
train_losses = []
train_accuracies = []

adversarial_loss = torch.nn.BCEWithLogitsLoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

best_model_wts = copy.deepcopy(discriminator.state_dict())
best_loss = 100

for epoch in range(NUM_EPOCHS):
    epoch_loss = 0.0
    epoch_acc = 0.0

    l_dataloader_iterator = iter(labeled_loader)

    for inputs_u, _  in tqdm(unlabeled_loader):
        try:
            inputs_l, targets_l = next(l_dataloader_iterator)
        except StopIteration:
            l_dataloader_iterator = iter(labeled_loader)
            inputs_l, targets_l = next(l_dataloader_iterator)
        
        inputs_l, targets_l = inputs_l.to(device), targets_l.to(device)
        inputs_u = inputs_u.to(device)
        batch_size = inputs_u.shape[0]

        generator.train()
        discriminator.train()

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)
        
        noise = torch.randn(batch_size, nz, device=device)
        generated = generator(noise)

        optimizer_G.zero_grad()
        optimizer_D.zero_grad()

        # TRAIN THE GENERATOR
        
        output, _ = discriminator(generated)
        gen_loss = adversarial_loss(output, valid)
        gen_loss.backward()
        optimizer_G.step()

        # TRAIN THE DISCRIMINATOR

        # Loss for labeled images
        _, label_aux = discriminator(inputs_l)
        d_label_loss = auxiliary_loss(label_aux, targets_l.squeeze().long())
        d_label_loss.backward()

        top1 = torch.argmax(label_aux, dim=1).long()
        ncorrect = torch.sum(top1 == targets_l)
        epoch_loss += d_label_loss.item()
        epoch_acc += ncorrect.item()

        # Loss for unlabeled images
        real_pred, _ = discriminator(inputs_u)
        d_real_loss = adversarial_loss(real_pred, valid)
        # d_real_loss.backward()

        # Loss for fake images
        fake_pred, _ = discriminator(generated.detach())
        d_fake_loss = adversarial_loss(fake_pred, fake)
        # d_fake_loss.backward()

        d_loss_adv = (d_real_loss + d_fake_loss) / 2
        d_loss_adv.backward()

        optimizer_D.step()
    
    generated = (generator(fixed_noise)+1.0)/2.0
    # vutils.save_image(generated.cpu().detach(), ('50generated_copy_%d.jpg' % epoch), normalize=True)
    
    val_loss, val_accuracy, val_top5 = test(discriminator, val_loader, 'val', criterion)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    epoch_loss /= len(unlabeled_loader.dataset)
    epoch_acc /= len(unlabeled_loader.dataset)

    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_acc)
    
    print(f"Epoch {epoch: >3}/{NUM_EPOCHS}, train loss: {epoch_loss:.2e}, train acc: {epoch_acc}")
    print(f"Epoch {epoch: >3}/{NUM_EPOCHS}, val loss: {val_loss:.2e}, val acc: {val_accuracy}, val top5: {val_top5}")
        
    if val_loss < best_loss:
        curr_patience = 0
        best_loss = val_loss
        best_classifier_wts = copy.deepcopy(discriminator.state_dict())
        torch.save(discriminator.state_dict(), 'Gan_D_copy50.pt')
        torch.save(generator.state_dict(), 'Gan_G_copy50.pt')
        print(test(discriminator, test_loader, 'test', criterion))
    else:
        curr_patience += 1
        if curr_patience == PATIENCE:
          break
    
    lr_scheduler_D.step()
    lr_scheduler_G.step()

plt.plot(range(len(val_losses)), np.array(val_losses))
plt.title('val loss')
plt.show()

plt.plot(range(len(val_accuracies)), np.array(val_accuracies))
plt.title('val acc')
plt.show()

plt.plot(range(len(train_losses)), np.array(train_losses))
plt.title('train loss')
plt.show()

plt.plot(range(len(train_accuracies)), np.array(train_accuracies))
plt.title('train acc')
plt.show()

print('Best VAL test accuracy: ', np.max(np.array(val_accuracies)),
      '% after ', np.argmax(np.array(val_accuracies)), ' training epochs')

100%|██████████| 704/704 [07:35<00:00,  1.54it/s]


Epoch   0/100, train loss: 1.89e-02, train acc: 7.379972443219699
Epoch   0/100, val loss: 1.19e-01, val acc: 0.19812075169932028, val top5: 0.6930227908836465
(0.07211635321959811, 0.31963788300835655, 0.8558495821727019)


100%|██████████| 704/704 [07:35<00:00,  1.55it/s]


Epoch   1/100, train loss: 1.21e-02, train acc: 7.508733721498733
Epoch   1/100, val loss: 2.59e-02, val acc: 0.5615753698520591, val top5: 0.9513194722111156
(0.024981832786522867, 0.6207520891364903, 0.9681058495821727)


100%|██████████| 704/704 [07:35<00:00,  1.55it/s]


...

Epoch  27/100, train loss: 2.56e-04, train acc: 7.955864705098004
Epoch  27/100, val loss: 6.40e-03, val acc: 0.8993402638944422, val top5: 0.9977009196321471


100%|██████████| 704/704 [07:35<00:00,  1.54it/s]


Epoch  28/100, train loss: 2.29e-04, train acc: 7.970332014756211
Epoch  28/100, val loss: 6.45e-03, val acc: 0.899640143942423, val top5: 0.9971011595361855


 44%|████▍     | 311/704 [03:21<04:14,  1.54it/s]


KeyboardInterrupt: ignored

In [ ]:
model = Discriminator(num_classes).to(device)
model.load_state_dict(best_classifier_wts)
test(model, test_loader, 'test', criterion)

(0.012285615655290051, 0.83008356545961, 0.9877437325905293)

In [ ]:
test(discriminator, test_loader, 'test', criterion)